In [1]:
import pandas as pd
import pickle

In [2]:
tags=pd.read_csv('Data/ml-small/tags.csv')
tags.drop(columns=["timestamp",],inplace=True)
ratings=pd.read_csv('Data/ml-small/ratings.csv')
ratings.drop(columns=["timestamp"],inplace=True)
genre=pd.read_csv('Data/ml-small/movies.csv')
plots=pd.read_csv('Data/ml-small/plots.csv')

In [3]:
tag_final=pd.DataFrame.from_dict(tags.groupby("movieId")['tag'].apply(set).apply(list))
tag_final.head()

,tag
movieId,
1,"[pixar, fun]"
2,"[Robin Williams, magic board game, game, fantasy]"
3,"[moldy, old]"
5,"[pregnancy, remake]"
7,[remake]


In [4]:
rating_final=pd.DataFrame.from_dict(ratings.groupby('movieId')['userId'].count())
rating_final.columns=['votes_counts']
rating_final['votes_avg']=pd.Series(ratings.groupby('movieId')['rating'].mean())
rating_final.head()

,votes_counts,votes_avg
movieId,,
1,215,3.920930
2,110,3.431818
3,52,3.259615
4,7,2.357143
5,49,3.071429


In [5]:
C=rating_final['votes_avg'].mean()
m=rating_final['votes_counts'].quantile(0.95)
def weighted_rating(x):
    v=x['votes_counts']
    R=x['votes_avg']
    return (v/(v+m)*R)+(m/(v+m))*C
rating_final['weighted_rating']=rating_final.apply(weighted_rating,axis=1)
rating_final.drop(columns=['votes_avg'],inplace=True)
rating_final.head()

,votes_counts,weighted_rating
movieId,,
1,215,3.802806
2,110,3.381115
3,52,3.260960
4,7,3.145094
5,49,3.164949


In [6]:
genre['release_year']=genre['title'].str.extract('(\d\d\d\d)', expand=True)
genre["title_movie"]= genre["title"].str.split("(", n = 1, expand = True)[0] 
genre['genres']=genre['genres'].apply(lambda x: x.split('|'))
genre.drop(columns=["title"],inplace=True)
genre.head()

,movieId,genres,release_year,title_movie
0,1,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,Toy Story
1,2,"[Adventure, Children, Fantasy]",1995,Jumanji
2,3,"[Comedy, Romance]",1995,Grumpier Old Men
3,4,"[Comedy, Drama, Romance]",1995,Waiting to Exhale
4,5,[Comedy],1995,Father of the Bride Part II


In [54]:
df_merge_col = pd.merge(tag_final, rating_final, on='movieId')

In [55]:
df_merge_col.head()

,tag,rating
movieId,,
1,"[fun, pixar]",3.920930
2,"[Robin Williams, magic board game, fantasy, game]",3.431818
3,"[old, moldy]",3.259615
5,"[pregnancy, remake]",3.071429
7,[remake],3.185185


In [56]:
merge_genre=pd.merge(df_merge_col, genre, on='movieId')

In [57]:
merge_final=pd.merge(merge_genre, plots, on='movieId')
merge_final.head()

,movieId,tag,rating,genres,release_year,title_movie,imdb,tmdb
0,1,"[fun, pixar]",3.920930,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,Toy Story,A cowboy doll is profoundly threatened and jea...,"Led by Woody, Andy's toys live happily in his ..."
1,2,"[Robin Williams, magic board game, fantasy, game]",3.431818,"[Adventure, Children, Fantasy]",1995,Jumanji,When two kids find and play a magical board ga...,When siblings Judy and Peter discover an encha...
2,3,"[old, moldy]",3.259615,"[Comedy, Romance]",1995,Grumpier Old Men,John and Max resolve to save their beloved bai...,A family wedding reignites the ancient feud be...
3,5,"[pregnancy, remake]",3.071429,[Comedy],1995,Father of the Bride Part II,George Banks must deal not only with the pregn...,Just when George Banks has recovered from his ...
4,7,[remake],3.185185,"[Comedy, Romance]",1995,Sabrina,An ugly duckling having undergone a remarkable...,An ugly duckling having undergone a remarkable...


In [58]:
merge_final.to_pickle('merged_data.pickle')

In [59]:
merge_final.to_csv('Data/ml-small/data.csv',index=False)